## Data Extraction Using Mixtral:7x8b

**This is a data extraction project in which we extract product attributes of individual product categories one by one.**
**Problem**: 
- There is a large set of HTML text in an excel sheet that has all the product attributes of our products. <br>
- The product attributes are not individually listed anywhere. <br>
- We need a dataset of product attributes in which various properties of each product are listed separately, in order to build a PIM system, and to use specific attributes to SEO-optimize our webshop texts. <br>
- The Challenge is that the text is not regular, different attributes come in a variety of formats from one product to the next. <br>
- Another challenge is that we do not have a comprehensive list of product properties and that also needs to be created along the way. <br>

**Solution**: 
1. The first solution was Text mining using Regular expressions. It was implemented on one product group by reading and analyzing product descriptions of many products and finding attributes and then generating the regex patterns to extract them. <br>
Successful, but took a lot of time and energy. 
2. The second solution was to use SpaCy and NLP methods to extract adjective and prepositional groups such as "mit Griff" or "mit Deckel" to then use them for attribute generation. <br>
This was a faster method than raw text mining, but the problem was a large number of false positives (adjectives that were not product attributes) and false negatives <br>
(items that were not adjectives or prepositional groups but were attributes of the product nevertheless). 
3. This lead to trying out LLMs. The first attempt was with the transformers library. However, running into Langchain and Ollama, I found them to be faster solutions. <br>
I used Ollama because it supported the Mixtral:7x8b model which is both compatible with structured outputs and also supports German language. <br>
The result is the following code that functions much better in extracting all the relevant data required for our products.

- Note: The prompt was an important part of the modeling which resulted in correct and coherent results that could be further processed.

In [1]:
## Importing libraries
import pandas as pd
import re
import json
from values import *
import ollama
import warnings
import os
from mistralai import Mistral
import time
warnings.filterwarnings('ignore')



In [2]:
## Loading Productgroup Number and its name
val = Values()
wg_list = val.wr_gr
wg_name = val.wr_name
client = val.client
model = val.model

In [3]:
## loading the csv file that has the product information for each warengruppe (previously generated using wr_folder_building.ipynb)
file = pd.read_excel(f'{val.parent_dir}{wg_list}_{wg_name}/{wg_list}_{wg_name}.xlsx',)
mined_text = file.copy()


In [4]:
## filtering only the necessary columns
mined_text = mined_text[['NUMMER','BANAME','BESCHREIBUNG']]
# mined_text = mined_text[:10]

In [5]:
allowed_keys = [
    # Basisinformationen
    "NUMMER",                         # Produkt ID
    "NAME",                           # KitchenAid Espressomaschine mit Kaffeemühle
    "PRODUKTART",                     # Espressomaschine, Vollautomat, Siebträgermaschine, Kapselmaschine, Filterkaffeemaschine
    "BRAND",                          # Hersteller, z. B. KitchenAid, Jura
    "MATERIAL",                       # Edelstahl, Kunststoff, Aluminium
    "FARBE",                          # Rot, Silber, Schwarz etc.
    "HERGESTELLT",                    # Deutschland, Italien

    # Maße & Gewicht
    "BREITE_cm",                      # 33,5 cm
    "TIEFE_cm",                       # 28 cm
    "HOEHE_cm",                       # 39,5 cm
    "ABMESSUNGEN_AUSSEN_cm",          # B/T/H zusammen, z. B. 33,5 x 28 x 39,5 cm
    "GEWICHT_kg",                     # 12 kg

    # Kapazitäten
    "KAPAZITAET_BOHNENBEHAELTER_g",   # 225 g
    "KAPAZITAET_WASSERTANK_l",        # 1,8 l
    "KAPAZITAET_MILCHBEHAELTER_l",    # 0,5 l
    "KAPAZITAET_TROPFSCHALE_l",       # 1 l
    "ANZAHL_TASSEN_PRO_BRUEHZYKLUS",  # 1 oder 2 Tassen

    # Mahlwerk & Kaffeezubereitung
    "MAHLWERK_ART",                   # Edelstahlmahlwerk, Kegelmahlwerk
    "ANZAHL_MAHLGRADE",               # 15 Mahlgradeinstellungen
    "ANPASSUNG_BRUEHSTAEKRE",         # Wahr/Falsch
    "ANPASSUNG_KAFFEEMENGE",          # Wahr/Falsch
    "ANPASSUNG_WASSERMENGE",          # Wahr/Falsch

    # Druck & Leistung
    "PUMPE_BAR",                      # 15 bar
    "SPANNUNG_VOLT",                  # 230 V
    "LEISTUNG_WATT",                  # 1500 W

    # Milchsystem
    "MILCHSYSTEM",                    # Milchlanze, Cappuccinatore, Milchkaraffe
    "MILCHAUFSCHAEUMER",              # Wahr/Falsch
    "HEISSWASSERFUNKTION",            # Wahr/Falsch

    # Steuerung & Bedienung
    "STEUERUNG",                      # Drehknöpfe, Touch, App
    "DISPLAY",                        # LED, LCD, TFT
    "PROGRAMME_AUTOMATIK",            # z. B. 10 Programme
    "SPEICHERPLAETZE",                # Anzahl individueller Nutzerprofile
    "AUTO_AUS",                       # Wahr/Falsch
    "TIMER",                          # Wahr/Falsch
    "KONTROLLLEUCHTE",                # Wahr/Falsch

    # Funktionen
    "FUNKTIONEN",                     # [Espresso, Cappuccino, Latte Macchiato, Americano]
    "HEISSWASSER",                    # Wahr/Falsch
    "TEEPROGRAMM",                    # Wahr/Falsch
    "DOPPELSCHUSS",                   # Wahr/Falsch (Doppio-Funktion)
    "VORBRUEHEN",                     # Wahr/Falsch
    "ENTKALKUNGSPROGRAMM",            # Wahr/Falsch
    "REINIGUNGSPROGRAMM",             # Wahr/Falsch
    "ABNEHMBARE_TEILE",               # Wahr/Falsch

    # Sicherheit
    "SICHERHEITSFUNKTIONEN",          # Überhitzungsschutz, Abschaltautomatik, Tropfstopp
    "KINDERSICHERUNG",                # Wahr/Falsch

    # Zubehör
    "ZUBEHOER",                       # Milchkännchen, Tamper, Reinigungsbürste
    "ZUBEHOER_ANZAHL",                # z. B. 1 Kännchen, 2 Filtereinsätze

    # Qualität & Zertifizierung
    "QUALITAET",                      # robustes Metallgehäuse, langlebige Technik
    "ZERTIFIKATE",                    # CE, geprüfte Sicherheit
    "PFLEGEHINWEIS",                  # Reinigung, Entkalkung, spülmaschinengeeignet

    # Weitere Eigenschaften
    "ANDERE_EIGENSCHAFTEN"            # ["integriertes Mahlwerk", "15 bar Pumpe", "Automatische Abschaltung"]
]


In [6]:
## function to clean responses from the LLM
def clean_response(response_text):
    response_text = response_text.replace(r"\\u00fc","ü")
    try:
        response_dict = json.loads(response_text)
        cleaned_dict = {key: value.strip() if isinstance(value, str) else value for key, value in response_dict.items()}
        return cleaned_dict

    except json.JSONDecodeError :

        print(f"Error: The response is not a valid JSON object.")
        print(response_text)

        return None

In [7]:
def clean_response(response_text):
    response_text = response_text.replace(r"\\u00fc", "ü")
    try:
        response_dict = json.loads(response_text)

        # Check if response_dict is a dictionary
        if isinstance(response_dict, dict):
            cleaned_dict = {key: value.strip() if isinstance(value, str) else value for key, value in response_dict.items()}
            return cleaned_dict

        # Check if response_dict is a list
        elif isinstance(response_dict, list):
            # Clean list elements if they are strings
            cleaned_list = [item.strip() if isinstance(item, str) else item for item in response_dict]
            return cleaned_list

        # If response_dict is neither dict nor list, return it as is
        else:
            return response_dict

    except json.JSONDecodeError:
        print(f"Error: The response is not a valid JSON object.")
        print(response_text)
        return None

# Original model - not modified

In [ ]:
### Loading the prompt and generating responses based on previous variables as well as extact instrctions
uncleaned_data = []
counter = 0
for id,name, net_beschreibung in zip(mined_text['NUMMER'],mined_text['BANAME'],mined_text['BESCHREIBUNG']):
    print(id,name)
    
    print(f"Product Number {counter + 1}")
    
    prompt = f"""
    [INST]
    Du extrahierst Produktattribute aus deutschem Fließtext.

    Gib **exakt ein** gültiges JSON-Objekt zurück, ohne zusätzlichen Text.

    Regeln:
    1) Verwende **ausschließlich** diese Schlüssel (exakt geschrieben, alle müssen vorhanden sein): {allowed_keys}
    2) Fülle fehlende Werte mit **"N/A"** (als String).
    3) Nutze nur Informationen aus **BESCHREIBUNG**. 
    4) Keine Schlüsse oder Vermutungen, nur direkte Informationen aus dem Text.
    5) Keine Unicode-Escapes (statt "\\u00e4" bitte "ä").
    6) Maße:
    - Grundformat: **B/T/H**
    - Falls Text **B/H/T** → interpretiere H als Tiefe, T als Höhe.
    - Falls Text **B/T/L** → interpretiere L als Höhe.
    - Ausgabeformat: **"B/T/H: <B> x <T> x <H>"** (ohne Einheit in der Zelle).
    7) Boolesche Eigenschaften werden als `"Wahr"` oder `"Falsch"` ausgegeben (kein True/False, kein Ja/Nein).
    8) Max. 50 Zeichen pro Wert – längere Werte werden abgeschnitten.
    9) Wenn Zahlen, Maße oder spezifische Werte im Text vorkommen, die keinem Schlüssel zugeordnet werden können, sammle sie in **"Andere Eigenschaften"** als Liste von Strings.
    10) Messeinheiten werden **vereinheitlicht**:
        - Leistung immer in **Watt (W)**, keine kW.
        - Spannung immer in **Volt (V)**.
        - Gewicht immer in **Kilogramm (kg)**.
        - Maße immer in **Zentimeter (cm)**.
        - Volumen immer in **Liter (l)**.
        - Pumpendruck immer in **bar**.
        Ausgabe: nur Zahlenwerte, Einheiten stehen im Spaltenkopf.
    11) Liefere **nur** das JSON-Objekt, ohne Kommentare oder Erklärungen.

    Eingaben:
    NUMMER: {id}
    NAME: {name}
    BESCHREIBUNG: {net_beschreibung}

    [/INST]
    """.strip()





    # Generate response using Mixtral 7x8b
    response = ollama.generate(model='mistral-small3.2:latest', prompt=prompt)
    # response = ollama.generate(model='bengt0/em_german_leo_mistral:Q8_0', prompt=prompt)

    # Clean up the response
    uncleaned_data.append(response['response'])
    print(response['response'])
    counter += 1

  384B01 Mobiler Elektro-Espressokocher_x000D_

Product Number 1
```json
{
  "NUMMER": "384B01",
  "NAME": "Mobiler Elektro-Espressokocher",
  "PRODUKTART": "Espressokocher",
  "BRAND": "N/A",
  "MATERIAL": "Edelstahl",
  "FARBE": "Matt gebürstet",
  "HERGESTELLT": "N/A",
  "BREITE_cm": "N/A",
  "TIEFE_cm": "N/A",
  "HOEHE_cm": "21",
  "ABMESSUNGEN_AUSSEN_cm": "N/A",
  "GEWICHT_kg": "1.2",
  "KAPAZITAET_BOHNENBEHAELTER_g": "N/A",
  "KAPAZITAET_WASSERTANK_l": "N/A",
  "KAPAZITAET_MILCHBEHAELTER_l": "N/A",
  "KAPAZITAET_TROPFSCHALE_l": "N/A",
  "ANZAHL_TASSEN_PRO_BRUEHZYKLUS": "3 oder 6",
  "MAHLWERK_ART": "N/A",
  "ANZAHL_MAHLGRADE": "N/A",
  "ANPASSUNG_BRUEHSTAEKRE": "N/A",
  "ANPASSUNG_KAFFEEMENGE": "N/A",
  "ANPASSUNG_WASSERMENGE": "N/A",
  "PUMPE_BAR": "N/A",
  "SPANNUNG_VOLT": "230",
  "LEISTUNG_WATT": "365",
  "MILCHSYSTEM": "N/A",
  "MILCHAUFSCHAEUMER": "N/A",
  "HEISSWASSERFUNKTION": "N/A",
  "STEUERUNG": "N/A",
  "DISPLAY": "N/A",
  "PROGRAMME_AUTOMATIK": "N/A",
  "SPEICHERPLAE

# Modified model, with Mistral API

In [ ]:
# ### Loading the prompt and generating responses based on previous variables as well as extact instrctions
# uncleaned_data = []

# for id,name, net_beschreibung in zip(mined_text['NUMMER'],mined_text['BANAME'],mined_text['BESCHREIBUNG']):
#     print(id,name)
    
#     prompt = f"""[INST]
#                 1. Extrahieren Sie die Produktattribute aus der folgenden Net_Beschreibung und geben Sie sie als gültiges JSON-Objekt in deutscher Sprache aus.
#                 2. Verwenden Sie NUR die allowed_keys = {allowed_keys} Schlüssel und ändern Sie diese nicht.
#                 3. Katalogue_beschreibung wird NUR berücksichtigt, wenn ein für das Modell erforderlicher Wert fehlt. Andernfalls überspringe sie.
#                 4. Nutzen Sie ALLE erlaubten Schlüssel im JSON-Objekt und setzen Sie nicht verfügbare Werte als "N/A".
#                 5. Beschränken Sie die Werte auf maximal 50 Zeichen.
#                 6. Verwenden Sie NUR die im Text vorhandenen Informationen, ohne Schlüsse oder Vermutungen.
#                 7. Das JSON-Objekt sollte KEINE ASCII-Kodierung enthalten.
#                 8. Überprüfen Sie doppelt, ob die Schlüssel alle genau gleich als "allowed_keys" sind.
#                 9. Achten Sie darauf, dass die Reihenfolge der Dimensionen B/T/H ist (Breite, Tiefe, Höhe).
#                 10. Werte sollten eindeutige Wörter sein und keine "Ja", "Nein", "True" oder "False". Beispiel: DECKEL = "Mit Deckel".
#                 11. Die Ausgabe sollte NUR das JSON-Objekt sein, ohne zusätzlichen Text außerhalb von geschweiften Klammern.
#                 12. Verwenden Sie ausschließlich Informationen aus dem aktuellen Text, ohne Vorwissen oder Ableitungen.
#                 [/INST]
                    
#                     NUMMER: {id}
#                     NAME: {name}
#                     Net_beschreibung: {net_beschreibung}
                  
#         """

#     messages = [
#         {
#             "role": "user",
#             "content": prompt
#         }
#     ]
#     chat_response = client.chat.complete(
#         model = model,
#         messages = messages,
#         response_format = {
#             "type": "json_object",
#         }
#     )

#     print(chat_response.choices[0].message.content)
# # # Generate response using Mixtral 7x8b
# #     response = ollama.generate(model='mixtral:latest', prompt=prompt)

# # # Clean up the response
#     uncleaned_data.append(chat_response.choices[0].message.content)
#     time.sleep(30)  # Pause for 20 seconds between requests to avoid rate limiting

In [ ]:
## cleaning one step further and appending responses to a new list
new_data = []
for id,item in enumerate(uncleaned_data):
    item = item.replace('```',"")
    item = item.replace('json',"")
    # print(clean_response(item))
    new_data.append(clean_response(item))
    

In [ ]:
## saving clean json-objects to a json file
with open(f'{val.parent_dir}{val.wr_gr}_{val.wr_name}/{val.wr_name}_{val.wr_gr}.json', 'w', encoding="utf-8") as f:
    json.dump(new_data, f, indent=4, ensure_ascii=False)    